

## Introduction: the acoustic  particulate cylinder

The propagation of waves in free space is governed by the 2D Helmholtz equation. We denote by $\mathrm V_n$ and $\mathrm U_n$ the cylindrical Bessel functions

$$
\tag{1}
 \mathrm V_n(k\mathbf r):=\mathrm J_n(kr)\mathrm e^{\mathrm in\theta} 
\quad\text{and}\quad
\mathrm U_n(k \mathbf r):= \mathrm H_n(k r)\mathrm e^{\mathrm in\theta}
$$

where $k$ is the background wavenumber and  $(r,\theta)$ are the polar coordinates of $\mathbf r$, ie $\mathbf r = (r\cos \theta,r\sin\theta)$, $\mathrm J_n$ and $\mathrm H_n$ are respectively the Bessel function of order zero and the Hankel function. 

We consider the scattering from a set of $J$ cylinders, referred to as particles, confined in a circular area of radius $R$. The  incident field $u_i$ can be decomposed in modes:

$$
\tag{2}
u_i(\mathbf r) =  \sum_{n=-\infty}^{+\infty}g_n\mathrm V_n(k\mathbf r),\quad g_n\in\mathbb{C}.
$$

Then the averaged scattered field $\langle u_s\rangle$ (over all possible configurations of the $J$ particles confined in the disc of radius $R$) can be written

$$
\tag{3}
\langle u_s\rangle(\mathbf r) =  \sum_{n=-\infty}^{+\infty} \langle \mathfrak F_n \rangle\mathrm H_n(k\mathbf r), \quad  \langle \mathfrak F_n \rangle = \mathrm T_ng_n
$$

where $\mathrm T_n\in\mathbb{C}$ is related to the effective T-matrix $\mathrm T_{n,m}$ of the effective cylinder of radius $R$. More precisely,

$$
\tag{4}
\mathrm T_{n,m} := \delta_{n,m} \mathrm T_n.
$$

With this definition, the scattering from the effective cylinder of radius $R$ can be summerized in matrix form 

$$
\tag{5}
 \langle \mathbf{F} \rangle = \mathbf T \mathbf g
$$

where we defined the vectors $\langle \mathbf{F} \rangle=( \langle \mathfrak F_n \rangle)_n$ and $\mathbf g = (g_n)_n$.

## Computation of the coefficients $\mathrm T_n$ 

First set the dimension and physics of the problem by defining the host medium:

In [ ]:
using EffectiveTMatrix
# 2D acoustic problem
dimension=2;                               
host_medium = Acoustic(dimension; ρ=1.0, c=1.0);

We consider a random particulate cylinder made of sound hard particles of radius 1.0. We also need to choose the volume fraction (density of particles). These information are stored in sp_MC:

In [ ]:
## particulate microstructure of the effective cylinder
particle = Particle(Acoustic(dimension; ρ=Inf, c=Inf),Circle(1.0)); # sound hard particles of radius 1.0 
sp_MC = Specie(particle; volume_fraction = .4) 

Finally, the cylinder is defined by it's radius and microstructure, ie an homogeneous material with the particles previously defined. For now, the homogeneous material has to be the same as the one of the host material:

In [ ]:
## define the effective cylinder
cylinder_radius = 20.0;
microstructure = Microstructure(host_medium,[sp_MC]);
cylinder = Material(Circle(cylinder_radius),microstructure);

To compute the coefficients $\mathrm T_n$ of the effective T-matrix, we need to specify the frequency $\omega$ and the number of coefficients to compute:

In [ ]:
## compute the coefficients of the T-matrix 
ω = .2; # frequency
N = 5;  # Tₙ for n ∈ [-N,N]
T = t_matrix(ω, host_medium, cylinder, basis_field_order=N);

<p align="center">
    <img
    src="T-matrix.png"
    alt="Alt text"
    title=""
    style="display: inline-block; margin: 0 auto; max-width: 200px">
</p>

The computation of the T-matrix uses the Effective Waves Method and is relatively fast. In Example 2 below, we show how to validate these results with Monte Carlo simulations. 

Once these coefficients are computed, the scattering from any incident field can be computed by using the decomposition (Eq. 2) of the incident field and formula (Eq. 3) for the scattered field. Note that the T-matrix does not depend on the incident field. Below is an example for computing the average pressure field resulting from an incident plane wave and a point source

In [ ]:
## Define incident planar wave and point source
# define the plane wave
plane_wave = plane_source(host_medium; direction = [1.0,0.0]);

# define the point source
x0 = [-4.5cylinder_radius,0.0];                    # origin of the point source
A = 1.0;                                           # amplitude of the point source
point_wave = point_source(host_medium, x0, A);

We then define a region where to compute the pressure field:

In [ ]:
# Define the incident plane wave
psource = plane_source(host_medium; direction = [1.0,0.0]);

# Define box where to compute the average scattered field 
P=Q=4*cylinder_radius;                   # bounding box size
bottomleft = [-P;-Q]; topright = [P;Q];
region = Box([bottomleft, topright]);    # bounding box

Finally the field is computed and plot with

In [ ]:
using Plots

## scattering from incident plane wave
us_plane_wave = average_scattered_field(ω, region, plane_wave, cylinder; basis_field_order=N);
plot(us_plane_wave, ω; field_apply=real,seriestype = :contour,c=:balance) 
plot!(title="Scattering from incident plave wave")

## scattering from point source
us_point_source = average_scattered_field(ω, region, point_wave, cylinder; basis_field_order=N);
plot(us_point_source,ω; field_apply=real,seriestype = :contour,c=:balance) 
plot!(title="Scattering from incident point source")

<p align="center">
  <img src="plane_wave_scattering.png" width="400px" />
  <img src="point_source_scattering.png" width="400px" /> 
</p>